In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
import numpy as np
import scipp as sc
import scippneutron as scn

from halt.io import cached_load

In [ ]:
# Data files from ownCloud
data = cached_load('../data/C60_n1e8.h5', 'EventData')
data

In [ ]:
scn.instrument_view(data, pixel_size=0.02)

In [ ]:
d = data.copy()
d.bins.coords['position'] = sc.bins_like(data, data.coords['position'])
d = d.bins.constituents['data']
x = d.coords['position'].fields.x.values
y = d.coords['position'].fields.y.values
z = d.coords['position'].fields.z.values
tof = d.data.values

In [ ]:
tof

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def find_normal(position):
    pca = PCA(n_components=3)
    pca.fit(position.values)
    return pca.components_[-1]

In [ ]:
def project_to_plane(da):
    position = data.coords['position']
    normal = find_normal(position)

    new_z_axis = sc.vector(value=normal / np.linalg.norm(normal))
    old_x_axis = np.array([1, 0, 0])
    new_y_axis = sc.vector(value=np.cross(old_x_axis, new_z_axis.value))
    new_x_axis = sc.vector(value=np.cross(new_z_axis.value, new_y_axis.value))

    new_origin = position.mean()
    p2 = position - new_origin

    new_position = sc.empty_like(position)
    new_position.fields.x = sc.dot(p2, new_x_axis)
    new_position.fields.y = sc.dot(p2, new_y_axis)
    new_position.fields.z = sc.scalar(0.0, unit=position.unit)
    
    new_data = data.copy(deep=False)
    new_data.coords['position'] = new_position
    return new_data

In [ ]:
data_in_plane = project_to_plane(data)

In [ ]:
scn.instrument_view(data_in_plane, pixel_size=0.02)